In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import pywt
from mne.time_frequency import tfr_multitaper
from sklearn.model_selection import train_test_split
import torch.cuda as cuda
from torch.autograd import Variable
import math

from utils import create_dataloader,train,bandpower
from dataset import EEG
import wandb
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
# Now do your import
from config import *

In [2]:

path = RECORED_PATH
run = 4
left_runs = [run]
right_runs = [run]
#8-10  10-12 13-20
band = [8,10]
subjects = [22]
#recorded eeg class
left_eeg = EEG(path, subjects, left_runs)
raw=left_eeg.data_to_raw()
right_eeg = EEG(path, subjects, right_runs)
right_raw = right_eeg.data_to_raw()

print("Raw done")
data, sf = raw.get_data(), raw.info['sfreq']

print(data.shape)
print(sf)


/Users/pongkornsettasompop/Desktop/work/Motor-Imagery/EEG-python/dataset/recorded_EEG
/Users/pongkornsettasompop/Desktop/work/Motor-Imagery/EEG-python/dataset/recorded_EEG
Raw done
(17, 48880)
125.0


In [3]:
#pick channel
# print(raw.info['ch_names'])
stim = ['T3','C3','STIM MARKERS']
raw = left_eeg.pickChannel(raw,stim)
right_raw = right_eeg.pickChannel(right_raw,stim)
# print(raw.info['ch_names'])
# print(len(raw.info['ch_names']))

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [4]:
X_l,y_l = left_eeg.raw_preprocess(raw,event_id=[1.0],rest_stage=False)
print(X_l.shape,y_l.shape)
print(X_l[0].shape)
bp = bandpower(X_l, sf, band)
mul_bp = bandpower(X_l,sf,[8,13],method='multitaper')
print('Average band power'+ str(bp))
print('Multitaper band'+ str(mul_bp))

handbp = bandpower(X_l[0:5],sf,band)
footbp = bandpower(X_l[6:10],sf,band)
print(handbp,footbp)

[428, 5317, 10206, 15095, 19984, 24873, 29762, 34651, 39540, 44429]
(10, 2, 875) (10,)
(2, 875)
Effective window size : 2.048 (s)
Average band power0.14643398387621648
Multitaper band0.05538886271249052
Effective window size : 2.048 (s)
Effective window size : 2.048 (s)
0.1374575866532473 0.16591794473979604


In [5]:
X_r,y_r = right_eeg.raw_preprocess(right_raw,event_id=[2.0],rest_stage=False)
print(X_r.shape,y_l.shape)
print(X_r[0:5].shape)
bp = bandpower(X_r, sf, band)
mul_bp = bandpower(X_r,sf,[8,13],method='multitaper')
print('Average band power'+ str(bp))
print('Multitaper band'+ str(mul_bp))

handbp = bandpower(X_r[0:5],sf,band)
footbp = bandpower(X_r[6:10],sf,band)
print(handbp,footbp)

[2873, 7762, 12651, 17540, 22429, 27318, 32207, 37095, 41984, 46874]
(10, 2, 875) (10,)
(5, 2, 875)
Effective window size : 2.048 (s)
Average band power0.11579221105264367
Multitaper band0.04216916695679131
Effective window size : 2.048 (s)
Effective window size : 2.048 (s)
0.1040430522383144 0.12368619514198056


In [6]:
# rest band
X_l,y_l = left_eeg.raw_preprocess(raw,event_id=[3.0],rest_stage=False)
print(X_l.shape,y_l.shape)
print(X_l[0].shape)
bp = bandpower(X_l, sf, band)
mul_bp = bandpower(X_l,sf,[8,13],method='multitaper')
print('Average band power'+ str(bp))
print('Multitaper band'+ str(mul_bp))

handbp = bandpower(X_l[0:5],sf,band)
footbp = bandpower(X_l[6:10],sf,band)
print(handbp,footbp)

[1683, 4128, 6573, 9017, 11462, 13906, 16351, 18795, 21240, 23684, 26129, 28573, 31018, 33462, 35906, 38351, 40795, 43240, 45685, 48129]
(20, 2, 875) (20,)
(2, 875)
Effective window size : 2.048 (s)
Average band power0.14011796304865842
Multitaper band0.050928523360378286
Effective window size : 2.048 (s)
Effective window size : 2.048 (s)
0.12093973194916185 0.12198425986187778


In [ ]:
X_l,y_l = left_eeg.raw_preprocess(raw,event_id=[1.0],rest_stage=False)
print(X_l.shape,y_l.shape)
X_r,y_r = right_eeg.raw_preprocess(right_raw,event_id=[2.0],rest_stage=False)
X = np.concatenate((X_l,X_r),axis=0)
y = np.concatenate((y_l,y_r),axis=0)
print(X.shape,y.shape)